# stats_size

> Calculate total dataset size

In [ ]:
path_input = "../data/test/parquet"
path_output = "../outs"
depth = 2
exclude = []
publish = False

In [ ]:
import datetime
import os

import pandas as pd

from rlmm.utils import *

In [ ]:
if not os.path.exists(os.path.abspath(path_input)):
    raise ValueError(f"Path does not exist: {path_input}")

In [ ]:
if not os.path.exists(os.path.abspath(path_output)):
    raise ValueError(f"Path does not exist: {path_outputh}")

In [ ]:
folders = walk_folders(path_input=path_input, depth=depth, exclude=exclude)

print(folders)

['../data/test/parquet/trades/ETHUSDT', '../data/test/parquet/book_snapshot_25/ETHUSDT']


In [ ]:
sizes = {}

for folder in folders:
    size = 0
    for f in os.listdir(folder):
        fp = f"{folder}/{f}"
        size += os.path.getsize(fp)

    folder = folder.replace(path_input + "/", "")
    sizes[folder] = round(size / (1024**3), 2)

print(sizes)

{'trades/ETHUSDT': 0.03, 'book_snapshot_25/ETHUSDT': 0.59}


In [ ]:
df = pd.DataFrame.from_dict(sizes, orient="index", columns=["GB"])

df

,GB
trades/ETHUSDT,0.03
book_snapshot_25/ETHUSDT,0.59


In [ ]:
df.reset_index(inplace=True)
df.rename(columns={"index": "path"}, inplace=True)

In [ ]:
new_cols = []
for i in range(0, depth):
    new_cols.append(f"level_{i}")

In [ ]:
print(new_cols)
print(df.columns)
print(df.head())

['level_0', 'level_1']
Index(['path', 'GB'], dtype='object')
                       path    GB
0            trades/ETHUSDT  0.03
1  book_snapshot_25/ETHUSDT  0.59


In [ ]:
df[new_cols] = df["path"].str.split("\/", expand=True).fillna("NA")
df = df[new_cols + ["GB"]]
df = df[df["GB"] != 0]

In [ ]:
if not df.empty:
    if publish and path_output and os.path.exists(path_output):
        ts = datetime.datetime.now().strftime("%y_%m_%d-%H_%M_%S")
        csv_path = f"{path_output}/dir_size_{ts}.csv"
        df.to_csv(csv_path, index=False)
    else:
        raise FileNotFoundError(f"Path {path_output} does not exist")

    print(df)
else:
    raise ValueError("dataframe empty")

            level_0  level_1    GB
0            trades  ETHUSDT  0.03
1  book_snapshot_25  ETHUSDT  0.59
